In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import torchaudio

In [3]:
from datasets import get_dataset
from configs.get_configs import get_config

In [4]:
import run_lib

In [5]:
class DotDict(dict):
    def __getattr__(self, key):
        return self[key]
    def __setattr__(self, key, val):
        if key in self.__dict__:
            self.__dict__[key] = val
        else:
            self[key] = val

In [6]:
from librosa.feature.inverse import db_to_power, mel_to_audio
import soundfile as sf

In [7]:
from configs.default_audio_configs import get_mels_64, get_mels_128

mel_cfg = get_mels_128()

sample_rate = mel_cfg.sample_rate
nfft = mel_cfg.nfft
hop_length = mel_cfg.hop_length

In [8]:
samples_path = "evaluation/128_deep/sampling_params/ckpt_270000/"
output_path = "evaluation/128_deep/sampling_params/audio/"
sample_files = os.listdir(samples_path)

for sample in sample_files:
    sample_name = sample.split(".")[0]
    mel_dat = np.load(f"evaluation/128_deep/sampling_params/ckpt_270000/{sample_name}.npz")["samples"]
    mel_data = mel_dat.squeeze()
    # reshape to -80 to 0 db range from librosa standard
    #mel_data /= mel_data.max()
    mel_data = np.clip(mel_data, 0.0, 1.0)
    mel_data *= 80
    mel_data -= 80

    mel_data = db_to_power(mel_data)
    audio = mel_to_audio(
        M=mel_data,
        sr=sample_rate,
        n_fft=nfft,
        hop_length=hop_length,
        win_length=hop_length * 4,
        center=True,
        power=1,
        n_iter=32,
        fmin=20.0,
        fmax=sample_rate / 2.0,
        pad_mode="reflect",
        norm='slaney',
        htk=True
    )
    
    sf.write(f'{output_path}{sample_name}.wav', audio, sample_rate, 'PCM_24')

/cluster/home/tshpakov/.local/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)
/cluster/home/tshpakov/.local/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


### Audio

In [9]:
from IPython.display import Audio
audio_iter = iter(os.listdir(output_path))

In [17]:
audio_file = next(audio_iter)
print(audio_file)
Audio(f"{output_path}{audio_file}", autoplay=True)

samples_7.wav
